#  A new model - K-Nearest Neighbors 💨🤯 


 [K-Nearest Neighbors (or KNN)](https://scikit-learn.org/0.24/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) es un modelo basado en la distancia que se puede usar tanto para la regresión (predecir un número) como para la clasificación (predecir una categoría)."
]

**¿Y la mejor parte?** ¡Los pasos son exactamente los mismos para el modelo KNN que para la regresión lineal 😎😎

 1. Comenzemos por importar  las bibliotecas de Python necesarias.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

Pongamos y leamos los datos de **CSV** en un  DataFrame

In [3]:
churn = pd.read_csv('datos/churn.csv')
churn

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,608,1,41,1,83807.86,1,0,1,112542.58,0
1,502,1,42,8,159660.80,3,1,0,113931.57,1
2,850,1,43,2,125510.82,1,1,1,79084.10,0
3,645,0,44,8,113755.78,2,1,0,149756.71,1
4,376,1,29,4,115046.74,4,1,0,119346.88,1
...,...,...,...,...,...,...,...,...,...,...
6378,597,1,53,4,88381.21,1,1,0,69384.71,1
6379,644,0,28,7,155060.41,1,1,0,29179.52,0
6380,516,0,35,10,57369.61,1,1,1,101699.77,0
6381,772,0,42,3,75075.31,2,1,0,92888.52,1


In [6]:
from sklearn.neighbors import KNeighborsClassifier # Importamos el modelo de Scikit-learn:

Inicializamos el modelo y eligimos una cantidad de **n_neighbors** para compararlos:

In [11]:
model = KNeighborsClassifier(n_neighbors=3)

Creamos nuestras `entradas` y `salidas`. Llamandolos `x` e `y`:

In [8]:
x = churn.drop(["Exited"], axis="columns") # dejando la columna de salida para crear las entradas (características)
y = churn["Exited"]

Revisamos la  **x**  e **y** a continuación 👇

In [13]:
x

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,608,1,41,1,83807.86,1,0,1,112542.58
1,502,1,42,8,159660.80,3,1,0,113931.57
2,850,1,43,2,125510.82,1,1,1,79084.10
3,645,0,44,8,113755.78,2,1,0,149756.71
4,376,1,29,4,115046.74,4,1,0,119346.88
...,...,...,...,...,...,...,...,...,...
6378,597,1,53,4,88381.21,1,1,0,69384.71
6379,644,0,28,7,155060.41,1,1,0,29179.52
6380,516,0,35,10,57369.61,1,1,1,101699.77
6381,772,0,42,3,75075.31,2,1,0,92888.52


In [14]:
y

0       0
1       1
2       0
3       1
4       1
       ..
6378    1
6379    0
6380    0
6381    1
6382    0
Name: Exited, Length: 6383, dtype: int64

💡Consejo: intente ajustar el número  - n_neighbors - arriba hasta que obtenga el mejor resultado.

In [15]:
model.fit(x,y) # Modelo de entrenamiento

KNeighborsClassifier(n_neighbors=3)

In [16]:
model.score(x,y) # resultado del modelo de entrenamiento

0.8171706094313019

Esta métrica de puntuación de tiempo es la precisión: cuántas predicciones acertó el modelo en nuestro conjunto de datos.



¿Cómo es tu puntuación? Bastante genial para pocas lineas de codigo, ¿verdad? 🤩Bueno...

## ¡Hemos estado haciendo trampa! 😳

Hemos estado puntuando el modelo con los mismos datos con los que se entrenó, ¡demasiado fácil! Eso se llama fuga de datos(**data leakage**).

![title](Img/Data_Leakage.jpeg)

La biblioteca Scikit-learn nos salva de nuevo: importemos y usemos el [`train_test_split` metodo](https://scikit-learn.org/0.24/modules/generated/sklearn.model_selection.train_test_split.html)

In [20]:
from sklearn.model_selection import train_test_split


El método `train_test_split` nos brinda todos los conjuntos de datos que necesitamos: entradas y salidas tanto para entrenamiento como para prueba. Entonces creamos simultáneamente cuatro nuevas variables para almacenar eso.

In [21]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)


###  ¡Esta vez sin trampas! 🚀

Ahora que tenemos conjuntos de datos de entrenamiento y prueba, debe **iniciar** un nuevo modelo, **entrenar** y **puntuar** nuevamente con los conjuntos de datos correctos.

In [23]:
model = KNeighborsClassifier(n_neighbors=3)

model.fit(xtrain, ytrain) # usando solo los datos de entrenamiento
model.score(xtest, ytest) # usar datos de prueba no vistos para calificar

0.6851996867658575

### ¡Predicción! 🚀

La predicción también funciona exactamente como el modelo anterior, así que le permitimos hacerlo. Creamos un cliente de ejemplo que puede ajustar.

*Nota: el orden de las características es:*

`['CreditScore', 'Gender', 'Edad', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']`

In [27]:
customer = [[608.0, 1.0, 31.0, 20.0, 81207.86, 2.0, 1.0, 0.0, 111142.58]]


In [28]:
model.predict(customer)

array([0], dtype=int64)

In [29]:
model.predict_proba(customer)

array([[0.66666667, 0.33333333]])

### Explainability? Already harder with KNN model 😓

We need to use a [feature permutation](https://scikit-learn.org/0.24/modules/generated/sklearn.inspection.permutation_importance.html) method provided by `Scikit-learn`.

This method runs the model scoring many times, by changing (*permutating*) one feature at a time, to see which one causes the most change in the target.

In [31]:
from sklearn.inspection import permutation_importance

permutation_score = permutation_importance(model, xtrain, ytrain, n_repeats=10)

np.vstack((x.columns, permutation_score.importances_mean)).T

array([['CreditScore', 7.833920877399203e-05],
       ['Gender', 0.0],
       ['Age', 0.0],
       ['Tenure', 0.0],
       ['Balance', 0.13037602820211516],
       ['NumOfProducts', 0.0],
       ['HasCrCard', 0.0],
       ['IsActiveMember', 0.0],
       ['EstimatedSalary', 0.12906384645515084]], dtype=object)

# Congratulations on your first ML models, heroes! 🦸‍♀️🦸‍♂️

## 🕵️‍♀️ Going further? (Optional Challenge) KNN only on categorical columns!

You probably noticed that the **categorical columns** - such as `HasCrCard` or `NumOfProducts` - have virtually no influence on the above model.

It might just be that they are strongly outweighed by the other columns, like `Balance` or `EstimatedSalary`. It would be a good idea **create a new model** *only* for the categorical columns.

### Your turn - build your own model! 🚀

Create a new KNN model **from scratch** and repeat the steps all the way to prediction but this time **only on the categorical columns**. We listed them below:

`['NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Gender']`

#### **💡Tip:**  Don't be shy to re-use from before and adapt it to the current challenge!

1. Create our `x` (inputs) and `y` (output) datasets

In [32]:
x = churn.drop(['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary'], axis='columns')
  # or instead of dropping columns, we can also just select the ones we need:
x = churn[['NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Gender']]

y = churn.Exited

2. Split our data into training and testing data with `train_test_split`   (*P.S. no need to `import` a second time*)

In [33]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

3. Initialize a KNN model. Pick the number of neighbors `n_neighbors` you like 🙂

In [34]:
classifier = KNeighborsClassifier(n_neighbors=3)

Siempre es una buena idea hacer algo de exploración visual primero.🤨😶‍🌫️

En este caso, nos importa si el cliente `salió` o **abandonó**. Así que podemos intentar trazar las diferentes características (entradas) manteniendo la `salida` como el diferenciador de color (matiz).

In [10]:
sns.scatterplot(data=churn, x='PICK A COLUMN', y='PICK ANOTHER COLUMN', hue='Exited')

ValueError: Could not interpret value `PICK A COLUMN` for parameter `x`